This code is to classify the selected examples automatically for those criteria that allow automatic classification.

In [1]:
import pandas as pd
import xlrd

In [2]:
# opens the excel sheet and reads it
# excel sheet needs to be in the same working directory as this file
# requires installation of xlrd >= 1.0.0 for Excel support (conversion to .xlsx required by openpyxl)

file = 'selected_rows_sorted.xls'
data = pd.ExcelFile(file)

 # returns the all the sheet names within the excel file
print(data.sheet_names)

['Sheet1']


In [7]:
# parses the sheet into a data frame to show the column structure within the file

data_frame = data.parse('Sheet1')
data_frame.info

# only shows the first 10 rows
data_frame.head(10)

,recherche,occurrence,wh,int,other,segment,enregistrement,transcription,locuteur,debut (ms),fin (ms),debut (hh:mm:ss),fin (hh:mm:ss)
0,Mot exact,comment,NaN,NaN,NaN,comment vous voyez ça ?,ESLO1_ENT_001,ESLO1_ENT_001_C,OU,302683,303581,0.003495,0.003507
1,Mot exact,comment,NaN,NaN,NaN,et comment est-ce que ce contrôle devrait se ?,ESLO1_ENT_001,ESLO1_ENT_001_C,OU,2973744,2976606,0.034410,0.034444
2,Mot exact,comment,NaN,NaN,NaN,comment ça ?,ESLO1_ENT_002,ESLO1_ENT_002_C,JR,148486,149249,0.001713,0.001725
3,Mot exact,comment,NaN,NaN,NaN,comment dirais-je ? malpolis malhonnêtes oui,ESLO1_ENT_002,ESLO1_ENT_002_C,DE744,699597,702487,0.008090,0.008125
4,Mot exact,comment,NaN,NaN,NaN,votre profession ça s'appelle comment ?,ESLO1_ENT_002,ESLO1_ENT_002_C,JR,785852,787593,0.009086,0.009109
5,Mot exact,comment,NaN,NaN,NaN,c'est triste comment est-ce que vous expl- vou...,ESLO1_ENT_002,ESLO1_ENT_002_C,JR,1486866,1489410,0.017199,0.017234
6,Mot exact,comment,NaN,NaN,NaN,aussi bien au point de vue euh comment dirais-...,ESLO1_ENT_002,ESLO1_ENT_002_C,DE744,4271706,4283592,0.049433,0.049572
7,Mot exact,comment,NaN,NaN,NaN,comment les Maisons de Culture en général ?,ESLO1_ENT_002,ESLO1_ENT_002_C,JR,4337408,4339130,0.050197,0.050220
8,Mot exact,comment,NaN,NaN,NaN,est-ce que je peux vous demander euh comment v...,ESLO1_ENT_003,ESLO1_ENT_003_C,JR,1087017,1099316,0.012581,0.012720
9,Mot exact,comment,NaN,NaN,NaN,comment ?,ESLO1_ENT_003,ESLO1_ENT_003_C,DJ39,1366017,1366517,0.015810,0.015810


In [8]:
# reads in the spreadsheet data

workbook = xlrd.open_workbook('selected_rows_sorted.xls')
sheet = workbook.sheet_by_name('Sheet1')

# gets the first sheet
sheet_1 = workbook.sheet_by_index(0)

row_count = sheet.nrows
col_count = sheet.ncols
print(f'Total rows: {row_count}\nTotal columns: {col_count}')

XLRDError: Excel xlsx file; not supported